In [1]:
from to_verb import *
from factuality_nominalizations import *
import sys, os, catvar

fact_corpus_path = "resources/unified_factuality"
factuality_dev = os.path.join(fact_corpus_path, "dev.conll")

In [2]:
ff = lambda b: df_from_table_string(b, col_names=COL_NAMES)
dfs=list(loadLineSplitFiles(factuality_dev, ff))

In [12]:
sentences = [' '.join(df['tok']) for df in dfs]
tokSents =  [list(df['tok']) for df in dfs]
find_nominalizations = lambda df: df[(df['pos'].isin(["NN", "NNP", "NNS"])) & (df['factuality']!="_")]
count_nominalizations = lambda df: find_nominalizations(df).shape[0]
find_non_nominalization = lambda df: df[(df['pos'].isin(["NN", "NNP", "NNS"])) & (df['factuality']=="_")]
count_non_nominalization = lambda df: find_non_nominalizations(df).shape[0]

# verbalize using both WordNet and CatVar
def get_verb_forms(nn):
    wordnet_verbs = convert_pos(nn, WN_NOUN, WN_VERB)
    catvar_verbs = catvariate(nn)
    # sort by distance
    vrbs = [v for v,w in results_by_edit_distance(nn, wordnet_verbs + catvar_verbs)]
    if vrbs:
        return vrbs
    else:
        return [nn]

In [ ]:
# prompts for qasrl-crowdsourcing variant 
prompts = [(tokSent, int(nom_row['tok-id']), get_verb_forms(nom_row['tok'])[0])
           for tokSent, df in zip(tokSents, dfs) 
           for i,nom_row in find_nominalizations(df).iterrows()]
noms = [p[0][p[1]] for p in prompts]

In [13]:
# prompts for MTurk UI nom. identification task
prompts = [{"tok_sent": tokSent,
            "target_index": int(nom_row['tok-id']),
            "verb_forms": get_verb_forms(nom_row['tok'])}
           for tokSent, df in zip(tokSents, dfs) 
           for i,nom_row in find_nominalizations(df).iterrows()]
noms = [p["tok_sent"][p["target_index"]] for p in prompts]

In [5]:
dict(enumerate([(' '.join(p[0][p[1]-1:p[1]+2]), p[2]) for p in prompts]))

{0: ('their battle to', u'battle'),
 1: ("'s return to", u'return'),
 2: ("a move '", u'move'),
 3: ('cause damage not', u'damage'),
 4: ('a threat to', u'menace'),
 5: ("'s remarks .", u'remark'),
 6: ('those worries during', u'worry'),
 7: ('television appearance Sunday', u'appear'),
 8: ('a speech to', u'speak'),
 9: ('those concerns had', u'concern'),
 10: ('of complaints about', u'complain'),
 11: ('program trading .', u'trade'),
 12: ('Program trading is', u'trade'),
 13: ('program trading ,', u'trade'),
 14: ('guided buying and', u'buy'),
 15: ('and selling of', u'sell'),
 16: ('offsetting trades in', u'trade'),
 17: ('program trading is', u'trade'),
 18: ('', u'estimate'),
 19: ('Program trading also', u'trade'),
 20: ('program trading it', u'trade'),
 21: ('price moves ,', u'move'),
 22: ('program trading is', u'trade'),
 23: ('undercutting efforts to', u'exert'),
 24: ('these problems are', u'trouble'),
 25: ('tumultuous slide in', u'slide'),
 26: ('the practice against', u'p

In [6]:
import json
# dump first 10 to be prompts
json.dump(prompts[:15], open("nom_prompts.json", "w"), indent=2)

In [7]:
dict(enumerate([(' '.join(p[0][p[1]-5:p[1]+7]), p[2]) for p in prompts]))

{0: ("him if they lose their battle to block the boy 's return", u'battle'),
 1: ("to block the boy 's return to Cuba .", u'return'),
 2: ("the possibility of such a move ' ' which could cause damage", u'move'),
 3: ("' ' which could cause damage not only to a child ,", u'damage'),
 4: ('had no information on a threat to Elian and was unaware of', u'menace'),
 5: ("was unaware of Castro 's remarks .", u'remark'),
 6: ('Cuban leader first raised those worries during another live television appearance Sunday',
  u'worry'),
 7: ('worries during another live television appearance Sunday night during a speech to',
  u'appear'),
 8: ('appearance Sunday night during a speech to about 700 Cuban university students',
  u'speak'),
 9: ('that Castro revealed that those concerns had been shared directly with the',
  u'concern'),
 10: ('to the swelling chorus of complaints about program trading .',
  u'complain'),
 11: ('chorus of complaints about program trading .', u'trade'),
 12: ('strong practi

In [8]:
problematic_wn_nomin = {
    # question with verb is not natural
    32: ('trading activity dwindled', u'act'),
    # verb not derivationally related to noun (even though meaning is presevred) - while there is a sutable verb:
    4: ('a threat to', u'menace'),
    101: ('In anticipation of', u'expect'),
    70: ('a turnaround next', u'reverse'),
    68: ('million loss for', u'pass'),
    71: ('the loss ,', u'pass'),
    121: ('a gain from', u'profit'),
    184: ('the alliance .', u'align'),
    230: ('quarter loss of', u'pass'),
    # not the correct source-verb (but same "root")
    132: ('the revolution that', u'revolutionize'),
    # not a nominalization (no verb is suitable)
    24: ('these problems are', u'trouble'),
    79: ('procurement scandal .', u'scandalise'),
    103: ('transition period .', u'point'),
    181: ('50th anniversary .', 'anniversary'),
    213: ('world events .', u'effect'),
    218: ('no problem with', u'trouble'),
    # light-verb constructions nouns (e.g. "effort" - "make an effort" would have worked, but no verb is suitable) 
    23: ('undercutting efforts to', u'exert'),
    38: ("'s efforts to", u'exert'),
    208: ('those efforts .', u'exert'),
    # not a verb!
    104: ('new series of', 'series'),
    112: ('a deadline StatesWest', 'deadline'),
    118: ('a restructuring program', 'restructuring'),
    214: ('increasingly terrorism knows', 'terrorism'),
    235: ('first salvo in', u'volley'),
    # two-word verbs 
    46: ('massive selloffs ,', u'sell_off'),
    48: ('share turnover for', u'turn_over'),
    75: ('the layoffs of', u'lay_off')
}
prb_prompts = {i:p for i,p in enumerate(prompts) if i in problematic_wn_nomin}
{i:(' '.join(p[0][p[1]-3:p[1]+3]), p[2]) for i,p in prb_prompts.iteritems()}

{4: ('information on a threat to Elian', u'menace'),
 23: ('trading is undercutting efforts to woo', u'exert'),
 24: ('Some of these problems are neither', u'trouble'),
 32: ('market and trading activity dwindled .', u'act'),
 38: ("the market 's efforts to bring", u'exert'),
 46: ('market after massive selloffs , market', u'sell_off'),
 48: ('average daily share turnover for October', u'turn_over'),
 68: ('$ 648.2 million loss for the', u'pass'),
 70: ('prepare for a turnaround next year', u'reverse'),
 71: ('size of the loss , coupled', u'pass'),
 75: ('related to the layoffs of 8,000', u'lay_off'),
 79: ('defense - procurement scandal .', u'scandalise'),
 101: ('', u'expect'),
 103: ('through the transition period .', u'point'),
 104: ('of a new series of Cilcorp', 'series'),
 112: ('yesterday , a deadline StatesWest had', 'deadline'),
 118: ('account for a restructuring program .', 'restructuring'),
 121: ('part by a gain from the', u'profit'),
 132: ('fighters in the revolution th

In [9]:
[(' '.join(p[0]), p[0][p[1]], [v[0] for v in verbalize(p[0][p[1]])]) for p in prompts[80:]]

[('Unisys also noted that it paid $ 78.8 million in taxes during the quarter , even though tax payments normally would be minimal in a quarter that produced such a big loss .',
  'payments',
  [u'pay', u'requite', u'defray']),
 ('Unisys also noted that it paid $ 78.8 million in taxes during the quarter , even though tax payments normally would be minimal in a quarter that produced such a big loss .',
  'loss',
  [u'pass', u'go', u'exit', u'depart', u'expire']),
 ('The tax payments will leave Unisys with $ 225 million in loss carry - forwards that will cut tax payments in future quarters .',
  'payments',
  [u'pay', u'requite', u'defray']),
 ('In addition , Unisys said it reduced computer inventories a further $ 100 million during the quarter , leaving it within $ 100 million of its goal of a reduction of $ 500 million by the end of the year .',
  'reduction',
  [u'reduce', u'decrease', u'simplify', u'step_down']),
 ('Still , Unisys said its European business was weak during the quarter

In [10]:
prompts[:10][0]

(['President',
  'Fidel',
  'Castro',
  'said',
  'Tuesday',
  'his',
  'government',
  'told',
  'U.S.',
  'officials',
  'it',
  'is',
  'worried',
  'that',
  'Miami',
  '-',
  'based',
  'exiles',
  'might',
  'take',
  'Elian',
  'Gonzalez',
  'to',
  'another',
  'country',
  'or',
  'even',
  'harm',
  'him',
  'if',
  'they',
  'lose',
  'their',
  'battle',
  'to',
  'block',
  'the',
  'boy',
  "'s",
  'return',
  'to',
  'Cuba',
  '.'],
 33,
 u'battle')